<a href="https://colab.research.google.com/github/TheNewLearn/bigdata_hw2/blob/main/bigdata_hw2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [84]:
!apt-get -y install openjdk-8-jre-headless
!pip install pyspark
from pyspark.sql import SparkSession
from pyspark import SparkContext
from google.colab import drive
from pyspark import SparkContext
from pyspark import SparkConf
from pyspark.sql import SQLContext
spark = SparkSession.builder.master("local").config("spark.driver.memory", "15g").getOrCreate()
sc = SparkContext.getOrCreate()

Reading package lists... Done
Building dependency tree       
Reading state information... Done
openjdk-8-jre-headless is already the newest version (8u292-b10-0ubuntu1~18.04).
0 upgraded, 0 newly installed, 0 to remove and 37 not upgraded.


In [51]:
drive.mount("/content/drive")
path = "/content/drive/My Drive/News_Final.csv"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [91]:
sqlContext = SQLContext(sc)
dataset = sqlContext.read.format("csv").options(header=True, inferSchema='True', quotes="\"",escape="\"").load(path);

dataset.show(2)


/usr/local/lib/python3.7/dist-packages/pyspark/sql/context.py:79: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning


+-------+--------------------+--------------------+---------+-------+-------------------+-----------------+-------------------+--------+----------+--------+
| IDLink|               Title|            Headline|   Source|  Topic|        PublishDate|   SentimentTitle|  SentimentHeadline|Facebook|GooglePlus|LinkedIn|
+-------+--------------------+--------------------+---------+-------+-------------------+-----------------+-------------------+--------+----------+--------+
|99248.0|Obama Lays Wreath...|Obama Lays Wreath...|USA TODAY|  obama|2002-04-02 00:00:00|              0.0|-0.0533001790889026|      -1|        -1|      -1|
|10423.0|A Look at the Hea...|Tim Haywood, inve...|Bloomberg|economy|2008-09-20 00:00:00|0.208333333333333| -0.156385810542806|      -1|        -1|      -1|
+-------+--------------------+--------------------+---------+-------+-------------------+-----------------+-------------------+--------+----------+--------+
only showing top 2 rows



In [36]:
news_title = dataset.rdd.map(lambda x:(x['Title'],1)).reduceByKey(lambda a,b:a+b)
#sorted(news_title.collect(),key=lambda x:x[1],reverse=True)

news_headline = dataset.rdd.map(lambda x:(x['Headline'],1)).reduceByKey(lambda a,b:a+b)
#sorted(news_headline.collect(),key=lambda x:x[1],reverse=True)

news_perday_title = dataset.rdd.map(lambda x:((x['PublishDate'][0:10],x['Title']),1)).reduceByKey(lambda x,y:x+y)
#sorted(news_perday_title.collect(),key=(lambda x:x[1]),reverse=True)

#news_perday_headline = dataset.rdd.map(lambda x:((x['PublishDate'][0:10],x['Headline']),1)).reduceByKey(lambda x,y:x+y)
#sorted(news_perday_headline.collect(),key=(lambda x:x[1]),reverse=True)


news_per_topic_title = dataset.rdd.map(lambda x:((x['Topic'],x['Title']),1)).reduceByKey(lambda x,y:x+y)
#sorted(news_per_topic_title.collect(),key=(lambda x:x[0][0]),reverse=True)

news_per_topic_headline = dataset.rdd.map(lambda x:((x['Topic'],x['Headline']),1)).reduceByKey(lambda x,y:x+y)
sorted(news_per_topic_headline.collect(),key=(lambda x:x[0][0]),reverse=True)
  



[(('palestine',
   'RAMALLAH, February 25, 2016 (WAFA) - Palestine Liberation Organization’s Sectretary-General Saeb Erekat Thursday expressed his concerns over Kenyan President Uhuru Kenyatta’s visit to Jerusalem and the Jordan Valley. '),
  5),
 (('palestine',
   'A Hancock County man lost a barn early Monday morning that had been in his family more than 100 years. '),
  1),
 (('palestine',
   'RAMALLAH, February 29, 2016 (WAFA) - The Government of Japan contributed $38.21 million to the United Nations Relief and Works Agency for Palestine Refugees in the Near East (UNRWA). '),
  5),
 (('palestine',
   'Forced evictions of Palestinian farmers from their land in the Jordan Valley and from the Hebron district """ to make room for more settlements'),
  1),
 (('palestine',
   "Palestine's flag will fly at UN headquarters after the general assembly overwhelmingly approved a Palestinian resolution, infuriating Israel with"),
  1),
 (('palestine',
   'Tomorrow, the Palestinian flag will be 

In [121]:
from pyspark.sql.functions import col, lit
dataset = sqlContext.read.format("csv").options(header=True, inferSchema='True', quotes="\"",escape="\"").load(path)
Facebook_pathlist = ["/content/drive/My Drive/Facebook/Facebook_Microsoft.csv","/content/drive/My Drive/Facebook/Facebook_Obama.csv","/content/drive/My Drive/Facebook/Facebook_Palestine.csv"]

fe_dc = sqlContext.read.format("csv").options(header=True, inferSchema='True', quotes="\"",escape="\"").load("/content/drive/My Drive/Facebook/Facebook_Economy.csv")
for i in Facebook_pathlist:
  fe_df = sqlContext.read.format("csv").options(header=True, inferSchema='True', quotes="\"",escape="\"").load(i)
  fe_dc = fe_dc.union(fe_df)


fulldataset = dataset.join(fe_dc).where(dataset['IDLink']== fe_dc['IDLink'])
rowMean  = (sum(col(x) for x in fulldataset.columns[13:]) / 144).alias("mean")
fulldataset.withColumn('mean',rowMean).show()


  




+------+--------------------+--------------------+--------------------+-------+-------------------+-------------------+-------------------+--------+----------+--------+------+---+---+---+---+---+---+---+---+---+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+--------------------+
|IDLink|               Title|            Headli